In [1]:
from music21 import converter, instrument, note, chord, stream
from keras.models import load_model
import numpy as np
import pickle
import random

##Retrieving Notes

In [2]:
with open("notes",'rb') as filepath:
    notes = pickle.load(filepath)

##Mappings

In [3]:
pitchnames = sorted(set(notes))
element_to_integer = dict((element, num) for num,element in enumerate(pitchnames))
integer_to_element = dict((num, element) for num,element in enumerate(pitchnames))

##Preprocessing notes

In [4]:
test_input = []
sequence_length = 100
number_of_examples = len(notes)-sequence_length
number_of_classes = len(pitchnames) 

for i in range(number_of_examples):
    seq_in = notes[i:i+sequence_length] #list of strings
    test_input.append([element_to_integer[s] for s in seq_in])

##Load Saved Model

In [5]:
model = load_model("weights.hdf5")

##Predictions

In [6]:
start = np.random.randint(number_of_examples-1)
pattern = test_input[start]
pattern_output = []

for note_index in range(50):
    print(note_index)
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(number_of_classes)
    prediction = model.predict(prediction_input,verbose=0)
    idx = np.argmax(prediction)
    result = integer_to_element[idx]
    pattern_output.append(result)
    pattern.append(idx)
    pattern = pattern[1:]



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


##Creating MIDI FILE from prediction

In [7]:
offset =0
output_notes = []

for pattern in pattern_output:
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else :
        new_note = note.Note(pattern)
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset
        output_notes.append(new_note)
    offset+=random.random();

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp="test_output3.mid")
    

'test_output3.mid'

In [8]:
midi_stream.show('midi')